In [1]:
import numpy as np
import pymysql
import configparser
import json
import time

In [2]:
ip = "139.199.102.41"
config = configparser.RawConfigParser()
config.readfp(open('./settings.cfg'))

dbname = config.get('jx3bla', 'username')
dbpwd = config.get('jx3bla', 'password')
db = pymysql.connect(ip,dbname,dbpwd,"jx3bla",port=3306,charset='utf8')
cursor = db.cursor()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: This method will be removed in future versions.  Use 'parser.read_file()' instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [3]:
edition = "3.5.0"
mapdetail = "25人英雄范阳夜变"

sql = """SELECT * FROM XiangZhiStat WHERE edition='3.5.0' AND mapdetail='25人英雄范阳夜变'"""
cursor.execute(sql)
result = cursor.fetchall()

In [4]:
allstat = {}
allstat["score"] = []
for line in result:
    stat = json.loads(line[7].replace("'", '"'))
    for key in stat:
        if key not in allstat:
            allstat[key] = [stat[key]]
        else:
            allstat[key].append(stat[key])
    allstat["score"].append(line[2])

In [28]:
avgstat = {}
medstat = {}
maxstat = {}
minstat = {}
numstat = {}

for key in allstat:
    data = allstat[key]
    if type(data[0]) == type(0) or type(data[0]) == type(0.0):
        nData = np.array(data)
        avgstat[key] = np.mean(nData)
        medstat[key] = np.median(nData)
        maxstat[key] = np.max(nData)
        minstat[key] = np.min(nData)
        numstat[key] = nData.shape[0]
    elif key in ["healTable", "dpsTable", "spareRateList", "rateTable"]:
        bossData = {}
        for record in data:
            for line in record:
                if line[0] not in bossData:
                    bossData[line[0]] = [line[1:]]
                else:
                    bossData[line[0]].append(line[1:])
        avgstat[key] = []
        medstat[key] = []
        maxstat[key] = []
        minstat[key] = []
        numstat[key] = []
        for line in bossData:
            nData = np.array(bossData[line])
            avgstat[key].append([line] + list(np.mean(nData, axis=0)))
            medstat[key].append([line] + list(np.median(nData, axis=0)))
            maxstat[key].append([line] + list(np.max(nData, axis=0)))
            minstat[key].append([line] + list(np.min(nData, axis=0)))
            numstat[key].append(nData.shape[0])
    elif key in ["breakList", "rateList"]:
        if key == "breakList":
            subkey = "bossBreakDict"
        elif key == "rateList":
            subkey = "bossRateDict"
        occData = {}
        for i in range(len(data)):
            for line in data[i]:
                if line[3] not in occData:
                    occData[line[3]] = [[line[1]] + allstat[subkey][i][line[0]]]
                else:
                    occData[line[3]].append([line[1]] + allstat[subkey][i][line[0]])
        avgstat[key] = []
        medstat[key] = []
        maxstat[key] = []
        minstat[key] = []
        numstat[key] = []
        avgstat[subkey] = {}
        medstat[subkey] = {}
        maxstat[subkey] = {}
        minstat[subkey] = {}
        numstat[subkey] = {}
        for line in occData:
            nData = np.array(occData[line])
            avgstat[key].append([line, np.mean(nData[:, 0]), "平均数", line])
            medstat[key].append([line, np.median(nData[:, 0]), "中位数", line])
            maxstat[key].append([line, np.max(nData[:, 0]), "最大值", line])
            minstat[key].append([line, np.min(nData[:, 0]), "最小值", line])
            numstat[key].append(nData.shape[0])
            avgstat[subkey][line] = list(np.mean(nData[:, 1:], axis=0))
            medstat[subkey][line] = list(np.median(nData[:, 1:], axis=0))
            maxstat[subkey][line] = list(np.max(nData[:, 1:], axis=0))
            minstat[subkey][line] = list(np.min(nData[:, 1:], axis=0))
            numstat[subkey][line] = nData.shape[0]
            
    elif key in ["npcHealList"]:
        occData = {}
        for i in range(len(data)):
            for line in data[i]:
                if line[3] not in occData:
                    occData[line[3]] = [line[1]]
                else:
                    occData[line[3]].append(line[1])
        avgstat[key] = []
        medstat[key] = []
        maxstat[key] = []
        minstat[key] = []
        numstat[key] = []
        for line in occData:
            nData = np.array(occData[line])
            avgstat[key].append([line, np.mean(nData), "平均数", line])
            medstat[key].append([line, np.median(nData), "中位数", line])
            maxstat[key].append([line, np.max(nData), "最大值", line])
            minstat[key].append([line, np.min(nData), "最小值", line])
            numstat[key].append(nData.shape[0])
    elif key in ["healList"]:
        occData = {}
        for i in range(len(data)):
            for line in data[i]:
                if len(line) <= 6:
                    continue
                if line[1] not in occData:
                    occData[line[1]] = [line[2:]]
                else:
                    occData[line[1]].append(line[2:])
        avgstat[key] = []
        medstat[key] = []
        maxstat[key] = []
        minstat[key] = []
        numstat[key] = []
        for line in occData:
            nData = np.array(occData[line])
            avgstat[key].append(["平均数", line] + list(np.mean(nData, axis = 0)))
            medstat[key].append(["中位数", line] + list(np.median(nData, axis = 0)))
            maxstat[key].append(["最大值", line] + list(np.max(nData, axis = 0)))
            minstat[key].append(["最小值", line] + list(np.min(nData, axis = 0)))
            numstat[key].append(nData.shape[0])
    elif key in ["hitCount"]:
        hitData = {}
        for line in data:
            for skillkey in line:
                if skillkey not in hitData:
                    hitData[skillkey] = [line[skillkey]]
                else:
                    hitData[skillkey].append(line[skillkey])
        avgstat[key] = {}
        medstat[key] = {}
        maxstat[key] = {}
        minstat[key] = {}
        numstat[key] = {}
        for skillkey in hitData:
            nData = np.array(hitData[skillkey])
            avgstat[key][skillkey] = np.mean(nData)
            medstat[key][skillkey] = np.median(nData)
            maxstat[key][skillkey] = np.max(nData)
            minstat[key][skillkey] = np.min(nData)
            numstat[key][skillkey] = nData.shape[0]
    elif key in ["hardBOSS", "hardNPC", "maxSingleBreakName", "maxSingleRateName"]:
        avgstat[key] = data[0]
        medstat[key] = data[0]
        maxstat[key] = data[0]
        minstat[key] = data[0]
        numstat[key] = data[0]
    elif key in ["maxDpsTable"]:
        occData = {}
        for i in range(len(data)):
            if allstat["maxDpsName"][i] not in ["厌夜", "藤原武裔"]:
                continue
            avgstat["maxDpsName"] = allstat["maxDpsName"][i]
            medstat["maxDpsName"] = allstat["maxDpsName"][i]
            maxstat["maxDpsName"] = allstat["maxDpsName"][i]
            minstat["maxDpsName"] = allstat["maxDpsName"][i]
            numstat["maxDpsName"] = allstat["maxDpsName"][i]
            for line in data[i]:
                if line[3] not in occData:
                    occData[line[3]] = [line[1]]
                else:
                    occData[line[3]].append(line[1])
        avgstat[key] = []
        medstat[key] = []
        maxstat[key] = []
        minstat[key] = []
        numstat[key] = []
        for line in occData:
            nData = np.array(occData[line])
            avgstat[key].append([line, np.mean(nData), "平均数", line])
            medstat[key].append([line, np.median(nData), "中位数", line])
            maxstat[key].append([line, np.max(nData), "最大值", line])
            minstat[key].append([line, np.min(nData), "最小值", line])
            numstat[key].append(nData.shape[0])
    elif key in ["bossBreakDict", "bossRateDict", "printTable", "maxDpsName"]:
        pass
    else:
        pass
        print(key)
        print(data[0])
avgstat["printTable"] = []
medstat["printTable"] = []
maxstat["printTable"] = []
minstat["printTable"] = []
numstat["printTable"] = []
print(avgstat)

for key in ["maxDpsTable", "rateList", "breakList"]:
    avgstat[key].sort(key=lambda x:-x[1])
    medstat[key].sort(key=lambda x:-x[1])
    maxstat[key].sort(key=lambda x:-x[1])
    minstat[key].sort(key=lambda x:-x[1])


{'score': 89.23076923076923, 'rateTable': [['周贽', 0.7914066333169886], ['厌夜', 0.8998612680217744], ['迟驻', 0.5136068508431105], ['白某', 0.6825315551770025], ['安小逢', 0.6662800747837796], ['狼牙精锐', 0.7780138531718797]], 'maxDpsName': '厌夜', 'maxDpsTable': [['22', 105518.14967610614, '平均数', '22'], ['1', 51545.324794884764, '平均数', '1'], ['9', 79708.29142311263, '平均数', '9'], ['6', 35988.06401033271, '平均数', '6'], ['4', 65020.34932723554, '平均数', '4'], ['5', 54810.12930242369, '平均数', '5'], ['8', 70592.41804299034, '平均数', '8'], ['21', 77698.91993555789, '平均数', '21'], ['7', 71297.1728901754, '平均数', '7'], ['10', 26795.98421862204, '平均数', '10'], ['3', 35639.8465703671, '平均数', '3'], ['2', 65958.36685809298, '平均数', '2'], ['23', 64098.44660184537, '平均数', '23'], ['25', 78103.9182248314, '平均数', '25'], ['24', 72231.7835806368, '平均数', '24']], 'numheal': 31562590.945054945, 'maxDpsRank': 1.0384615384615385, 'maxEqualDPS': 1.5952127815678567, 'hardBOSS': '安小逢', 'npcHealList': [['5', 135549.77990430623, '平均数', 

In [29]:
if mapdetail == "25人英雄范阳夜变":
    mapabbr = "yd"
elif mapdetail == "25人英雄敖龙岛":
    mapabbr = "ald"

avgInfo = {"server": "统计数据", "id": "平均数", "score": avgstat["score"], "battledate": time.strftime("%Y-%m-%d", time.localtime()), 
           "mapdetail": mapdetail, "edition": edition, "hash": "%saverage"%mapabbr, "statistics": avgstat}
medInfo = {"server": "统计数据", "id": "中位数", "score": medstat["score"], "battledate": time.strftime("%Y-%m-%d", time.localtime()), 
           "mapdetail": mapdetail, "edition": edition, "hash": "%smedian"%mapabbr, "statistics": medstat}
maxInfo = {"server": "统计数据", "id": "最大值", "score": maxstat["score"], "battledate": time.strftime("%Y-%m-%d", time.localtime()), 
           "mapdetail": mapdetail, "edition": edition, "hash": "%smaximum"%mapabbr, "statistics": maxstat}
minInfo = {"server": "统计数据", "id": "最小值", "score": minstat["score"], "battledate": time.strftime("%Y-%m-%d", time.localtime()), 
           "mapdetail": mapdetail, "edition": edition, "hash": "%sminimum"%mapabbr, "statistics": minstat}
numInfo = {"server": "统计数据", "id": "样本数", "score": numstat["score"], "battledate": time.strftime("%Y-%m-%d", time.localtime()), 
           "mapdetail": mapdetail, "edition": edition, "hash": "%snum"%mapabbr, "statistics": numstat}

In [8]:
from painter import XiangZhiPainter

painter = XiangZhiPainter(printint=0)
painter.paint(avgInfo, "average.png")
painter.paint(medInfo, "median.png")
painter.paint(maxInfo, "maximun.png")
painter.paint(minInfo, "minimum.png")

In [33]:
import json
import urllib.request

class MyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        if isinstance(obj, time):
            return obj.__str__()
        else:
            return super(NpEncoder, self).default(obj)

def upLoadData(info):
    Jdata = json.dumps(info, cls=MyEncoder)
    jpost = {'jdata': Jdata}
    jparse = urllib.parse.urlencode(jpost).encode('utf-8')
    resp = urllib.request.urlopen('http://139.199.102.41:8009/uploadXiangZhiData', data = jparse)
    res = json.load(resp)
    return res

res = upLoadData(avgInfo)
print(res)
res = upLoadData(medInfo)
print(res)
res = upLoadData(maxInfo)
print(res)
res = upLoadData(minInfo)
print(res)

{'num': 285, 'numOver': 90, 'result': 'dupid'}
{'num': 285, 'numOver': 144, 'result': 'dupid'}
{'num': 285, 'numOver': 266, 'result': 'success'}
{'num': 286, 'numOver': 1, 'result': 'success'}


In [31]:
print(maxInfo)

{'server': '统计数据', 'id': '最大值', 'score': 100, 'battledate': '2020-09-23', 'mapdetail': '25人英雄范阳夜变', 'edition': '3.5.0', 'hash': 'ydmaximum', 'statistics': {'score': 100, 'rateTable': [['周贽', 0.9628638537866396], ['厌夜', 0.9680685766492992], ['迟驻', 0.7644777520731318], ['白某', 0.8502291693492818], ['安小逢', 0.857025796131175], ['狼牙精锐', 0.8794934934798339]], 'maxDpsName': '厌夜', 'maxDpsTable': [['22', 153368.69649775108, '最大值', '22'], ['21', 134073.78747454355, '最大值', '21'], ['8', 122725.8688278695, '最大值', '8'], ['7', 120012.79064802699, '最大值', '7'], ['9', 119355.65363094829, '最大值', '9'], ['25', 113793.04531264471, '最大值', '25'], ['2', 112930.01299093067, '最大值', '2'], ['3', 109816.49000363576, '最大值', '3'], ['10', 109172.29234684742, '最大值', '10'], ['24', 106595.41989201418, '最大值', '24'], ['4', 102895.91381802401, '最大值', '4'], ['23', 102798.04419127644, '最大值', '23'], ['6', 102325.48761492998, '最大值', '6'], ['1', 91967.31240236801, '最大值', '1'], ['5', 87099.30871145906, '最大值', '5']], 'numheal': 775

In [32]:
maxInfo["score"] = int(maxInfo["score"])
minInfo["score"] = int(minInfo["score"])
maxInfo["statistics"]["score"] = int(maxInfo["score"])
minInfo["statistics"]["score"] = int(minInfo["score"])
testInfo = {}
stat = maxInfo["statistics"]
for key in stat:
    print(key)
    testInfo[key] = stat[key]
    Jdata = json.dumps(testInfo)

score
rateTable
maxDpsName
maxDpsTable
numheal
maxDpsRank
maxEqualDPS
hardBOSS
npcHealList


TypeError: Object of type int32 is not JSON serializable

In [24]:
print(type(maxInfo["statistics"]["numheal"]))



<class 'numpy.int32'>


In [27]:
type(maxInfo["statistics"]["numheal"]) == type(np.array([1])[0])

True